# Initialisation

In [611]:
import matplotlib
matplotlib.use('Qt5Agg')  # Utiliser le backend Qt5Agg pour Windows
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.optimize import curve_fit
import cv2
import numpy as np
import os
from PIL import Image
import lmfit
#import tifffile
#%matplotlib tk #pour Linux

In [612]:
import scipy.ndimage as ndimage
from sklearn.cluster import DBSCAN

# Importer fichier

In [613]:
path_to_tiff = os.path.join("..", "acquisition", "test_cond_rele_200ms_1im_gain=150_black=0.tiff")

tiff = Image.open(path_to_tiff)

# Nombre de frames pas vide

In [614]:
with Image.open(path_to_tiff) as img:
    frame_number = 0
    actual_frames = 0
    try:
        while True:
            frame_number += 1
            if np.sum(np.array(img)) != 0:
                actual_frames += 1
                
            img.seek(frame_number)
    except EOFError:
        print("All frames processed.")

actual_frames

All frames processed.


71

# 1re frame

**Le nombre de frames ignorés n'est pas pris en compte**

In [615]:
frame_index = 0
first_frame = 0
tiff.seek(frame_index)
original_image = np.array(tiff)

while np.sum(original_image) == 0:
    frame_index += 1
    tiff.seek(frame_index)
    original_image = np.array(tiff)
    first_frame += 1
    print(frame_index)




# Traitement d'image

In [616]:
clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
preprocessed = clahe.apply(original_image)

blurred = cv2.medianBlur(preprocessed, 115)
preprocessed2 = cv2.subtract(preprocessed, blurred)

# Apply Non-Local Means Denoising
img = cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)


# Sélection du point à tracker

In [617]:
def crop(img, x, y, crop_size_x, crop_size_y):
    x_start = int(x - crop_size_x // 2)
    x_end = int(x + crop_size_x // 2)
    y_start = int(y - crop_size_y // 2)
    y_end = int(y + crop_size_y // 2)

    return img[y_start:y_end, x_start:x_end]

In [618]:
# Display the image and let the user select a point interactively
fig, ax = plt.subplots()
ax.imshow(img, origin='lower', cmap='gray')  # Use 'gray' for better visibility of grayscale images
plt.title(f"Frame {frame_index}: Select a point")

# Ask for a point to be selected
print("Please click on the point you want to select.")
x, y = plt.ginput(1)[0]  # This will get the coordinates of the clicked point
print(f"Selected point: ({x}, {y})")
plt.close()

crop_sze = crop_sze_x = crop_sze_y = 50

Please click on the point you want to select.
Selected point: (795.4740259740262, 783.2012987012987)


Debogueur

In [619]:
def visionneur(frame, titre):
    plt.figure(figsize=(10, 5))
    plt.clf() 
    plt.imshow(frame, origin='lower', cmap='gray')
    plt.title(titre)
    plt.colorbar()  
    plt.show()

# Fit gaussien sur le point sélectionné

In [620]:
def prepare_data(x, y, z):
    return (x.flatten(), y.flatten()), z.flatten()

In [621]:
def gaussian_2d(xy, amplitude, x0, y0, sigma_x, sigma_y, offset):
    x, y = xy
    a = 1 / (2 * sigma_x**2)
    b = 1 / (2 * sigma_y**2)
    return offset + amplitude * np.exp(- (a * (x - x0)**2 + b * (y - y0)**2))

In [622]:
def localisateur_gaussien(intensity_grid, centre):
    x = np.arange(intensity_grid.shape[1])
    y = np.arange(intensity_grid.shape[0])
    X, Y = np.meshgrid(x, y)

    # Préparer les données pour le fit
    (xdata, ydata), zdata = prepare_data(X, Y, intensity_grid)
    model = lmfit.Model(gaussian_2d)
    max_idx = np.unravel_index(np.argmax(intensity_grid), intensity_grid.shape)
    initial_x0 = x[max_idx[1]]
    initial_y0 = y[max_idx[0]]

    # Définir les paramètres du modèle
    params = model.make_params(
        amplitude=np.max(intensity_grid),
        x0=initial_x0,
        y0=initial_y0,
        sigma_x=1,
        sigma_y=1,
        offset=2
    )

    # Ajouter des bornes sur les paramètres x0 et y0
    params['x0'].min = 0    # Limite inférieure pour x0
    params['x0'].max = intensity_grid.shape[1] - 1  # Limite supérieure pour x0
    params['y0'].min = 0    # Limite inférieure pour y0
    params['y0'].max = intensity_grid.shape[0] - 1  # Limite supérieure pour y0

    # Ajouter des bornes sur les paramètres sigma
    params['sigma_x'].min = 1  # Limite inférieure de sigma_x
    params['sigma_x'].max = 20 # Limite supérieure de sigma_x
    params['sigma_y'].min = 1  # Limite inférieure de sigma_y
    params['sigma_y'].max = 20 # Limite supérieure de sigma_y

    # Effectuer l'ajustement
    result = model.fit(zdata, params, xy=(xdata, ydata))
    
    x_position = result.params['x0'].value + centre[0] - crop_sze_x/4
    y_position = result.params['y0'].value + centre[1] - crop_sze_y/4

    return [x_position, y_position], result.params['sigma_x'].value, result.params['sigma_y'].value

# Process d'image (enlever le bruit)

**semble faire du trouble**

In [623]:
def denoise(image):
    clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
    preprocessed = clahe.apply(image)
    
    blurred = cv2.medianBlur(preprocessed, 115)
    preprocessed2 = cv2.subtract(preprocessed, blurred)
    
    return cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)

# Identifier la particule et donner sa nouvelle position

In [624]:
def is_within_bounds(position):
    """
    Vérifie si la position est à l'intérieur des limites du cadre (100 à 1300 en x et 100 à 980 en y).
    """
    x, y = position

    # Définir les bornes du cadre
    x_min, x_max = 100, 1300
    y_min, y_max = 100, 980

    # Vérifier si la position est dans les bornes du cadre
    if x_min <= x <= x_max and y_min <= y <= y_max:
        return True
    else:
        return False


In [625]:
def empreinte_digitale(image, threshold=24, eps=3, min_samples=1):
    # 1. Observer chaque particule approximativement
    #neighborhood_size = 3  
    #local_max = ndimage.maximum_filter(image, size=neighborhood_size)
    visionneur(image, f'particule choisie')

    maxima = (image > threshold)
    
    coordinates = np.column_stack(np.where(maxima))  
    if coordinates.shape[0] == 0:
        print("Aucune particule détectée.")
        return [], image

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)
    unique_labels = np.unique(labels)
    size=0

    # 2. Déterminer l'empreinte digitale de notre particule
    for label in unique_labels:
        if label != -1:  # -1 correspond au bruit dans DBSCAN
            cluster_points = coordinates[labels == label]
            plt.scatter(cluster_points[:, 1], cluster_points[:, 0], label=f'Particule {label}')

            if cluster_points.shape[0]>size:
                size = cluster_points.shape[0]
                luminosity = np.mean(image[cluster_points[:, 0], cluster_points[:, 1]])

    seuil=np.max(image)*0.5
    print(seuil)
    # 3. Afficher les résultats et filtrer les particules selon leur taille et luminosité
    plt.imshow(image, origin='lower', cmap='gray')
    plt.title(f"Empreinte digitale de la particule, lum={luminosity} et taille={size}")
    plt.legend()
    plt.colorbar()  
    plt.show()

    return size, luminosity, seuil

In [626]:
def detect_nbre_particles(image, threshold, eps=5, min_samples=1):
    # 1. Appliquer un filtre de voisinage pour détecter les maxima locaux
    neighborhood_size = 3  # Taille du voisinage pour détecter les maxima locaux
    local_max = ndimage.maximum_filter(image, size=neighborhood_size)
    # 2. Comparer l'image originale et les maxima locaux pour identifier les vrais maxima
    maxima = (image == local_max) & (image > threshold)

    # 3. Extraire les coordonnées des maxima (particules)
    coordinates = np.column_stack(np.where(maxima))  # Extraire les indices des pixels maximaux
    if coordinates.shape[0] == 0:
        print("Aucune particule détectée.")
        visionneur(image, f'détection du nombre de particules')

        return [], image

    # 4. Appliquer DBSCAN pour regrouper les particules proches (cluster les maxima détectés)
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)

    #5. Déterminer le nombre de particules
    unique_labels = np.unique(labels)
    nb_particules = len(unique_labels)
        
    # 6. Afficher les résultats et filtrer les particules selon leur taille et luminosité
    # plt.imshow(image, origin='lower', cmap='gray')
    # plt.title("Détection des particules")
    
    positions=[]
    for label in unique_labels:
        if label != -1:  # -1 correspond au bruit dans DBSCAN
            cluster_points = coordinates[labels == label]
            #plt.scatter(cluster_points[:, 1], cluster_points[:, 0], label=f'Particule {label}')
        positions.append((np.mean(cluster_points[:, 1]), np.mean(cluster_points[:, 0])))
        

    # plt.colorbar()  
    # plt.legend()
    # plt.show()

    # Retourner les coordonnées des particules filtrées
    return nb_particules, positions

In [627]:
def identificateur_particles(image, ref_size, ref_luminosity, threshold=25, eps=5, min_samples=1):
    # 1. Obtenir le plus d'info sur la particule
    maxima = (image > threshold)
    
    coordinates = np.column_stack(np.where(maxima))  
    if coordinates.shape[0] == 0:
        print("Mauvais re-crop empêche identification de la particule.")
        return [], image

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)
    unique_labels = np.unique(labels)

    # 2. Parcourir chaque particule et identifier laquelle est celle qu'on suit
    distances = []
    position_particules = []

    for label in unique_labels:
        if label != -1:  # -1 correspond au bruit dans DBSCAN
            cluster_points = coordinates[labels == label]
            
            size = cluster_points.shape[0]
            luminosity = np.mean(image[cluster_points[:, 0], cluster_points[:, 1]])
            size_distance = abs(size - ref_size)  
            luminosity_distance = abs(luminosity - ref_luminosity)  
            combined_distance = size_distance + luminosity_distance
            distances.append(combined_distance)
            position_particules.append((np.mean(cluster_points[:, 1]), np.mean(cluster_points[:, 0])))

    ecart = np.min(distances)
    position_best_particle = position_particules[np.argmin(distances)]  
    print(f'ecrat: {ecart}')
    # 3. Afficher les résultats et filtrer les particules selon leur taille et luminosité
    # plt.imshow(image, origin='lower', cmap='gray')
    # plt.title("Détection des particules")
    # plt.scatter(position_best_particle[0], position_best_particle[1], color='r', label=f'Particule sélectionnée')
    # plt.legend()
    # plt.colorbar()  
    # plt.show()

    return position_best_particle, ecart

# Faire le crop et fit

In [628]:
def particle_tracker_simple(image, x, y):
    image = denoise(image)

    cropped_img = crop(image, x, y, crop_sze_x, crop_sze_y)
    
    #Gérer plus qu'une particule
    cropped_img = np.array(cropped_img)
    max_index = np.argmax(cropped_img)
    max_coords = np.unravel_index(max_index, cropped_img.shape)

    nouveau_x = x - crop_sze_x // 2 + max_coords[1]
    nouveau_y = y - crop_sze_y // 2 + max_coords[0]
    
    second_crop = crop(image, nouveau_x, nouveau_y, crop_sze_x, crop_sze_y)    # Re-crop autour d'une seule particule

    result_fit = localisateur_gaussien(second_crop, [nouveau_x, nouveau_y])

    x_new, y_new = result_fit[0][0], result_fit[0][1]

    return [result_fit, cropped_img, (x_new, y_new), (result_fit[1], result_fit[2])]

In [629]:
#Placés ici pour pouvoir y faire référence dans particule_tracker
position_list=[[x,y]]
sigma_list = []
crop_frames = []
big_frames = []

In [630]:
def particle_tracker(imge, x, y, taille_initiale, luminosité_initiale, seuil):
    # 1. Ce qu'on voit près de l'ancienne position
    image = denoise(imge)

    cropped_img = crop(image, x, y, crop_sze_x, crop_sze_y)
    
    # 2. Déterminer combien il y a de particules et laquelle est la notre
    # Utiliser une reconnaissance de particules et si plus qu'une, alors clic pour choisir
    nb_part, coordonnées = detect_nbre_particles(cropped_img, seuil)
    #print(f'coordonnées des {nb_part} particules:{coordonnées}')

    if (nb_part == 0) or (nb_part == []):
        # Display the image and let the user select a point interactively
        fig, ax = plt.subplots()
        ax.imshow(denoise(img), origin='lower', cmap='gray') 
        plt.title(f"Frame {len(position_list)-1}: Select a point")

        # Ajouter une croix rouge à la position donnée
        cx, cy = position_list[len(position_list)-1]
        ax.plot(cx, cy, 'rx', markersize=10) 
        
        # Afficher l'image et demander à l'utilisateur de cliquer
        print("Please click on the point you want to select.")
        nouveau_x, nouveau_y = plt.ginput(1)[0]  # Attente du clic de l'utilisateur (1 point)
        print(f"Selected point: ({nouveau_x}, {nouveau_y})")
        
        # Fermer l'affichage après sélection
        plt.close()
    elif nb_part > 1:
        imperfections, emplacements = [] , []
        for num_particule in range(nb_part):
            x_identifier=x - crop_sze_x // 2 + coordonnées[num_particule][0]
            y_identifier=y - crop_sze_y // 2 + coordonnées[num_particule][1]
            crop_pour_identifier = crop(image, x_identifier, y_identifier, crop_sze_x//2, crop_sze_y//2)
            emplacement, ecart = identificateur_particles(crop_pour_identifier, taille_initiale, luminosité_initiale, seuil)
            imperfections.append(ecart)
            emplacements.append(emplacement)
        qualité = np.min(imperfections)
        print(f'écart:{qualité}')
        max_coords = emplacements[np.argmin(imperfections)]  
        nouveau_x = x_identifier - crop_sze_x // 4 + max_coords[0]
        nouveau_y = y_identifier - crop_sze_y // 4 + max_coords[1]
    else:
        cropped_img = np.array(cropped_img)
        max_index = np.argmax(cropped_img)
        max_coords = np.unravel_index(max_index, cropped_img.shape)

        nouveau_x = x - crop_sze_x // 2 + max_coords[1] #Oui c'est inversé à cause du unravel juste avant
        nouveau_y = y - crop_sze_y // 2 + max_coords[0]

    # Vérification s'il fout le camp 
    if np.sqrt((position_list[-1][0]-nouveau_x)**2+(position_list[-1][1]-nouveau_y)**2)>50:
        # Display the image and let the user select a point interactively
        fig, ax = plt.subplots()
        ax.imshow(denoise(img), origin='lower', cmap='gray') 
        plt.title(f"Frame {len(position_list)-1}: Problème d'identification, select a point")

        print('problème d''identification')
        # Ajouter une croix rouge à la position donnée
        cx, cy = position_list[-1]
        ax.plot(cx, cy, 'rx', markersize=10, label=f'Position estimée') 
        plt.legend()
        # Afficher l'image et demander à l'utilisateur de cliquer
        print("Please click on the point you want to select.")
        nouveau_x, nouveau_y = plt.ginput(1)[0]  # Attente du clic de l'utilisateur (1 point)
        print(f"Selected point: ({nouveau_x}, {nouveau_y})")
        
        # Fermer l'affichage après sélection
        plt.close()

    # 3. Crop atour de notre particule et fit dessus
    second_crop = crop(image, nouveau_x, nouveau_y, crop_sze_x//2, crop_sze_y//2)    # Re-crop autour d'une seule particule


    result_fit = localisateur_gaussien(second_crop, [nouveau_x, nouveau_y])

    x_new, y_new = result_fit[0][0], result_fit[0][1]

    # Affiche l'accord entre le fit gaussien et le blob (pour comparer)
    if len(position_list)==5:    
        x_list, y_list = zip(*position_list)
        x_list = list(x_list)  # Convertir en liste
        y_list = list(y_list)  # Convertir en liste        
        x_list.append(x_new)
        y_list.append(y_new)   
        plt.imshow(image, origin='lower', cmap='gray')#, extent=[x_position.min(), x_position.max(), y_position.min(), y_position.max()])
        plt.title("tracking de la particule")
        plt.scatter(x_list, y_list, color='b', label=f'Positions précédentes')
        plt.scatter(x_new, y_new, color='r', label=f'Position actuelle estimée par fit gaussien')
        plt.plot(x_list, y_list, color='b', linestyle='-', linewidth=1)
        plt.xlabel('Position horizontale (nombre de pixel)')
        plt.ylabel('Position veticale (nombre de pixel)')
        plt.legend()
        plt.show()

    return [result_fit, second_crop, (x_new, y_new), (result_fit[1], result_fit[2])]

# Main loop

In [631]:
taille_initiale, luminosité_initiale, seuil = empreinte_digitale(crop(img,x,y,35,35))
print(f'position initiale: {position_list}')

for i in range(first_frame+1,actual_frames):
    tiff.seek(i)
    if np.sum(np.array(tiff))==0:
        print(f'x={x}')
        position_list.append(np.array([np.nan,np.nan]))
        sigma_list.append(np.array([np.nan,np.nan]))
        print(f'frames vide:{i}')
    else:
        print(f'x={x}')
        data = particle_tracker(img, x, y, taille_initiale, luminosité_initiale, seuil)
        position_list.append(data[2])
        # x,y = position_list[i-first_frame+1] Valeur de First frame n'a pas d'impact apparemment
        x,y = position_list[i]
        sigma_list.append(data[3])
        tiff.seek(i)
        img = np.array(tiff)
        big_frames.append(img)
        crop_frames.append(data[1])
        if is_within_bounds(position_list[-1]):
            pass
        else:
            break
    print(f'liste des positions frame {i}:{position_list}')
    
    
position_list=position_list[1:] #On ignore le premier point cliqué car incertitude non définie

39.5
position initiale: [[795.4740259740262, 783.2012987012987]]
x=795.4740259740262
ecrat: 70.55214453584018
ecrat: 39.48363752615671
ecrat: 42.96796818071411
écart:39.48363752615671
liste des positions frame 1:[[795.4740259740262, 783.2012987012987], (795.5569524068497, 778.0680148527148)]
x=795.5569524068497
liste des positions frame 2:[[795.4740259740262, 783.2012987012987], (795.5569524068497, 778.0680148527148), (802.0601956794684, 775.9684664585202)]
x=802.0601956794684
liste des positions frame 3:[[795.4740259740262, 783.2012987012987], (795.5569524068497, 778.0680148527148), (802.0601956794684, 775.9684664585202), (805.3092524206794, 777.4067681091065)]
x=805.3092524206794
liste des positions frame 4:[[795.4740259740262, 783.2012987012987], (795.5569524068497, 778.0680148527148), (802.0601956794684, 775.9684664585202), (805.3092524206794, 777.4067681091065), (809.0863648227672, 775.2002724706055)]
x=809.0863648227672
liste des positions frame 5:[[795.4740259740262, 783.2012987

In [632]:
pixel_size = 3.45 # Taille absolue
f2 = 150  # Focale de L2
M_theo = 20  # Magnification of the objective
pxl = pixel_size / (f2 * M_theo / 160)  # Pixel size in um dans le plan de la particule

sigma_arr=np.array(sigma_list)* pxl * 10**(-6)
position_arr=np.array(position_list)* pxl * 10**(-6)
print(position_arr)

[[0.00014638 0.00014316]
 [0.00014758 0.00014278]
 [0.00014818 0.00014304]
 [0.00014887 0.00014264]
 [0.00015034 0.00014226]
 [       nan        nan]
 [       nan        nan]
 [0.0001506  0.00014195]
 [0.00015384 0.00014195]
 [0.0001542  0.00014197]
 [0.00015512 0.00014088]
 [0.00015569 0.00014184]
 [0.00015577 0.00014161]
 [0.00015755 0.0001428 ]
 [0.00015996 0.00014289]
 [0.0001609  0.00014244]
 [0.00016187 0.00014214]
 [0.00016331 0.0001431 ]
 [0.00016463 0.00014261]
 [0.00016521 0.00014381]
 [0.00016559 0.00014443]
 [0.00016554 0.00014522]
 [0.00016638 0.0001451 ]
 [0.00016779 0.00014579]
 [0.00016774 0.00014547]
 [0.00016752 0.00014547]
 [0.00016887 0.00014475]
 [0.00017056 0.00014511]
 [0.00017063 0.00014505]
 [0.00017158 0.00014551]
 [0.00017253 0.00014916]
 [0.00017383 0.00014626]
 [0.00017409 0.00014722]
 [0.00017413 0.00014703]
 [0.00017422 0.00014674]
 [0.00017546 0.00014815]
 [0.00017556 0.00014815]
 [0.00017756 0.00014899]
 [0.00017855 0.0001494 ]
 [0.00017896 0.00014968]


In [633]:
def calculate_msd_with_uncertainty(position_arr, delta_x, delta_y):
    """
    Calcule les MSD avec propagation des incertitudes analytiques.
    """

    msd = []
    uncertainties = []
    for d in range(1, len(position_arr)):
        diff_pairs = np.zeros_like(position_arr[d:])
        dx = np.zeros_like(delta_x[d:])
        dy = np.zeros_like(delta_y[d:])

        # Masque pour détecter les NaN
        nan_mask_pos = np.isnan(position_arr[d:]) | np.isnan(position_arr[:-d])
        nan_mask_delta = np.isnan(delta_x[d:]) | np.isnan(delta_x[:-d])
        
        # Calculer les différences uniquement où nan_mask est False
        valid_mask_pos = ~nan_mask_pos.any(axis=1)  # Inverser le masque pour obtenir les position_arr valides
        valid_mask_delta = ~nan_mask_delta

        diff_pairs[valid_mask_pos] = position_arr[d:][valid_mask_pos, :] - position_arr[:-d][valid_mask_pos, :]

        distances_squared = np.sum(diff_pairs**2, axis=1)
        msd.append(np.mean(distances_squared))  

        dx[valid_mask_delta] = delta_x[d:][valid_mask_delta] + delta_x[:-d][valid_mask_delta]
        dy[valid_mask_delta] = delta_y[d:][valid_mask_delta] + delta_y[:-d][valid_mask_delta]

        term_x = 2 * (diff_pairs[:, 0]**2) * (dx**2)
        term_y = 2 * (diff_pairs[:, 1]**2) * (dy**2)
        
        # Propagation des incertitudes
        total_uncertainty = np.mean(term_x + term_y)
        uncertainties.append(np.sqrt(total_uncertainty))

    return np.array(msd), np.array(uncertainties)

calculate_msd_with_uncertainty(position_arr, sigma_arr[:, 0], sigma_arr[:, 1])

(array([1.73589010e-12, 3.54074511e-12, 6.28538845e-12, 1.00176243e-11,
        1.44401011e-11, 2.10193043e-11, 2.83642363e-11, 3.58157893e-11,
        4.35335640e-11, 5.37032738e-11, 6.34678964e-11, 7.52310482e-11,
        8.79943968e-11, 1.01389279e-10, 1.15719942e-10, 1.30242250e-10,
        1.44823165e-10, 1.61391876e-10, 1.78442581e-10, 1.96093426e-10,
        2.12996059e-10, 2.33852012e-10, 2.53729955e-10, 2.73922334e-10,
        2.99307567e-10, 3.24950333e-10, 3.51102705e-10, 3.76173661e-10,
        4.01506791e-10, 4.28360201e-10, 4.53324968e-10, 4.75873515e-10,
        5.02405885e-10, 5.33334206e-10, 5.61427529e-10, 5.89069266e-10,
        6.20948363e-10, 6.45495069e-10, 7.09567236e-10, 7.43272359e-10,
        7.35878617e-10, 7.61187001e-10, 7.91738772e-10, 8.15292775e-10,
        8.38425069e-10, 9.29016778e-10, 9.65455091e-10, 9.95749367e-10,
        1.02957447e-09, 1.05726956e-09, 1.17347569e-09, 1.21921158e-09,
        1.16845073e-09, 1.18901575e-09, 1.21378211e-09, 1.251711

# Résultats

In [634]:
# Assuming msd_values is the result of your MSD calculation
msd_values, uncertainties = calculate_msd_with_uncertainty(position_arr, sigma_arr[:, 0], sigma_arr[:, 1])
print('coucou')
# Time intervals for the x-axis (assuming a uniform time step)
time_intervals = np.arange(1, len(msd_values) + 1) * 0.5

# Plotting the MSD
plt.figure(figsize=(8, 6))
plt.errorbar(time_intervals, msd_values, yerr=uncertainties, fmt='o', label='MSD', color='blue')

# Customize the plot
plt.title("MSD en fonction de l'intervalle de temps")
plt.xlabel("Intervalle de temps")
plt.ylabel("MSD")
plt.grid(True)
plt.legend()

# Show the plot
plt.show()

coucou


In [635]:
# Fonction quadratique pour l'ajustement
def quadratic(x, a, b, c):
    return a * x**2 + b * x + c

# Assuming msd_values is the result of your MSD calculation
msd_values, uncertainties = calculate_msd_with_uncertainty(position_arr, sigma_arr[:, 0], sigma_arr[:, 1])

cropped_msd = msd_values[:5]
cropped_inc = uncertainties[:5]

# Time intervals for the x-axis (assuming a uniform time step)
time_intervals = np.arange(1, len(cropped_msd) + 1)

# Perform a quadratic fit using curve_fit, which gives both coefficients and covariance matrix
popt, pcov = curve_fit(quadratic, time_intervals, cropped_msd, sigma=cropped_inc)

# popt contains the optimal coefficients
coeffs = popt

# pcov is the covariance matrix, which we can use to compute uncertainties
coeff_uncertainties = np.sqrt(np.diag(pcov))  # Incertitudes sur les coefficients sont les éléments diagonaux

# Affichage des coefficients séparément
print("Coefficients du polynôme ajusté :")
print(coeffs)

# Affichage de la matrice de covariance séparément
print("Matrice de covariance des coefficients :")
print(pcov)

# Création de la fonction quadratique de l'ajustement
quadratic_fit = np.poly1d(coeffs)

# Génération des valeurs ajustées pour afficher la courbe
fitted_msd = quadratic_fit(time_intervals)

# Affichage du MSD avec les barres d'erreur
plt.figure(figsize=(8, 6))
plt.errorbar(time_intervals, cropped_msd, yerr=cropped_inc, fmt='o', label='MSD', color='blue')
plt.plot(time_intervals, fitted_msd, label='Quadratic Fit', color='red', linestyle='--')

# Personnalisation du graphique
plt.title("MSD en fonction de l'intervalle de temps")
plt.xlabel("Intervalle de temps")
plt.ylabel("MSD")
plt.grid(True)
plt.legend()

# Afficher le graphique
plt.show()

# Affichage des incertitudes sur les coefficients
print(f"Incertitudes sur les coefficients : {coeff_uncertainties}")

# Calcul du coefficient de diffusion et de la taille de la particule
r = (4 * 1.38 * 10**(-23) * 300 / (6 * np.pi * 10**(-3) * coeffs[1]))  # Coefficient de diffusion
r_uncertainty = (4 * 1.38 * 10**(-23) * 300 * coeff_uncertainties[1] / (6 * np.pi * 10**(-3) * (coeffs[1])**2))  # Coefficient de diffusion

# Affichage de la taille de la particule avec incertitude
print(f"Taille de la particule (m): {r:.2e} m, avec une incertitude : {r_uncertainty:.2e}")


Coefficients du polynôme ajusté :
[4.54150377e-13 4.65813317e-13 8.11043829e-13]
Matrice de covariance des coefficients :
[[ 1.87896936e-28 -9.85856163e-28  9.41602234e-28]
 [-9.85856163e-28  5.43113083e-27 -5.44824562e-27]
 [ 9.41602234e-28 -5.44824562e-27  6.08757327e-27]]
Incertitudes sur les coefficients : [1.37075503e-14 7.36962063e-14 7.80229022e-14]
Taille de la particule (m): 1.89e-06 m, avec une incertitude : 2.98e-07


In [636]:
x_plt, y_plt = zip(*position_list)
# Create the plot
plt.figure(figsize=(10, 8))  # Optional: Adjust the figure size
plt.plot(x_plt, y_plt, marker='o', linestyle='-', color='b', label='Connected Points')

# Inverser l'axe Y
plt.gca().invert_yaxis()

# Set grid limits to match the 1440x1080 grid
plt.xlim(0, 1440)
plt.ylim(0, 1080)

# Add labels and title
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('2D Connected Points Plot')
plt.legend()

# Optional: Add grid lines
plt.grid(True)

# Afficher le graphique
plt.show()

## anim toutes les particules

In [637]:
fig, ax = plt.subplots()
img = ax.imshow(big_frames[0], origin='lower', cmap='gray', animated=True)


# Update function
def update(frame):
    img.set_array(frame)
    return img,
    
ani = animation.FuncAnimation(fig, update, frames=big_frames, interval=50, blit=True)
plt.show()

## anim crop

In [638]:
fig, ax = plt.subplots()
img = ax.imshow(crop_frames[0], origin='lower', cmap='gray', animated=True)


def update(frame):
    img.set_array(frame)
    return img,
    
ani = animation.FuncAnimation(fig, update, frames=crop_frames, interval=50, blit=True)
plt.show()

In [653]:
import cv2
import numpy as np

# Dimensions de la vidéo
width, height = 1440, 1080  # Résolution de la vidéo
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Codec (XVID, MJPG, etc.)
out = cv2.VideoWriter('output_video.avi', fourcc, 30.0, (width, height))  # Sortie vidéo

# Simulation d'images et positions
num_frames = len(big_frames)

for i in range(num_frames):
    print(i)
    # Créer une image vide (par exemple une image noire)
    frame = big_frames[i]
    
    # Vérifier si la position est valide (pas de NaN)

    position = position_list[i]
    if np.isnan(position[0]) or np.isnan(position[1]):
        position = 0,0       
    
    position = np.round(position).astype(int)  # Arrondir puis convertir en entier

    # Ajouter la particule sur l'image à la position correspondante
    cv2.circle(frame, position, 10, (0, 0, 255), -1)  # Dessiner un cercle rouge à la position

    # Ajouter l'image à la vidéo
    out.write(frame)

# Libérer la vidéo
out.release()

# Vérification : Si la vidéo est correctement enregistrée
print("Vidéo enregistrée sous 'output_video.avi'.")


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
Vidéo enregistrée sous 'output_video.avi'.
